In [ ]:
%reset -f 

In [ ]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
from tqdm import tqdm
import statistics as stat

#specify parameters
a = 4.19
b = 0.0639
sigma = 1.71
mu = 0.0259

a_tilde = a + (sigma**2)/2
b_tilde = (2*a*b)/(2*a + sigma**2)

B = 0.99
K = 0.96

#specify the initial condition (Y is V)
U_0 = np.log(0.9335)
Y_0 = 0.0649

#specify the numer of samples and range of tolerances
N = 1000
tolerances = [1/8, 1/10, 1/15, 1/20, 1/25, 1/50, 1/75, 1/100, 1/150, 1/200]

constants = {
    'a': a, 
    'b': b, 
    'vol': sigma, 
    'drift': mu,
    'a tilde': a_tilde, 
    'b tilde': b_tilde,
    'IV U': U_0,
    'IV V': Y_0,
    'barrier': B,
    'strike': K,
    'no_samples': N,
    'tolerance range': tolerances
}


# BM generator

In [ ]:
def BM(T, h):
#Define time discretization
    N = int(T/h)
    t = np.arange(0, T+h, h)

# Define the crude Brownian increments
    W_crude = np.zeros(N+1)
    dW_crude = np.zeros(N)
    W_crude[0]= 0
    for i in range(N):
        dW_crude[i] = np.random.normal(0, np.sqrt(h))
        W_crude[i+1] = W_crude[i] + dW_crude[i]

    BMoutputs = {
        'step size': h,
        'crude delta W': dW_crude
    }
    return BMoutputs

## Define refine function

In [ ]:
def refine(dW, h):
    B = np.random.normal(0, np.sqrt(h/4))
    half_h = h/2
    dW_1 = dW/2 - B
    dW_2 = dW/2 + B
    
    return dW_1, dW_2, half_h

## Define the function to create fine steps for Milstein

In [ ]:
def fine_step(y, u, constants, h, dWv, dWu, no_refines = 3):
    sigma = constants['vol']
    mu = constants['drift']
    a_tilde = constants['a tilde']
    b_tilde = constants['b tilde']
    if no_refines == 0:
        y_new = y + a_tilde*(b_tilde - y)*h + sigma*y*dWv + (1/2)*(sigma**2)*y*((dWv)**2)
        u_new = u + (mu - (1/2)*(y**2))*h + y*dWu + (1/2)*sigma*y*dWu*dWv
        return [y_new], [u_new], [dWv], [dWu]
    else:
        dWu_1, dWu_2, _ = refine(dWu, h)
        dWv_1, dWv_2, half_h_v = refine(dWv, h)
        y_new1, u_new1, dWv1 , dWu1= fine_step(y, u, constants, half_h_v, dWv_1, dWu_1, no_refines-1)
        y_new2, u_new2, dWv2, dWu2 = fine_step(y_new1[-1], u_new1[-1], constants, half_h_v, dWv_2, dWu_2, no_refines - 1)

        return y_new1 + y_new2, u_new1 + u_new2, dWv1 + dWv2, dWu1 + dWu2
    

# Define the function for adaptive Milstein method

In [ ]:
def milstein_adapt_iga(y, u, constants, tol, h, dWv, dWu,):
    sigma = constants['vol']
    mu = constants['drift']
    a_tilde = constants['a tilde']
    b_tilde = constants['b tilde']
    B = constants['barrier']
    c = tol
    
    if (B - np.exp(u) > c)  or (h <= 1/512):
        y_new = y + a_tilde*(b_tilde - y)*h + sigma*y*dWv + (1/2)*(sigma**2)*y*((dWv)**2)
        u_new = u + (mu - (1/2)*(y**2))*h + y*dWu + (1/2)*sigma*y*dWu*dWv
        return [y_new], [u_new], [dWv], [dWu], [h]
    else:
        dWu_1, dWu_2, _ = refine(dWu, h)
        dWv_1, dWv_2, half_h_v = refine(dWv, h)
        y_new1, u_new1, dWv1 , dWu1, h1 = milstein_adapt_iga(y, u, constants, c, half_h_v, dWv_1, dWu_1)
        y_new2, u_new2, dWv2, dWu2, h2 = milstein_adapt_iga(y_new1[-1], u_new1[-1], constants, c, half_h_v, dWv_2, dWu_2)
        return y_new1 + y_new2, u_new1 + u_new2, dWv1 + dWv2, dWu1 + dWu2, h1 + h2


## Run the Milstein with adaptive step sizes for different tolerances 

In [ ]:
T = 1
h_0 = 1/2 * T
u_0 = constants['IV U']
y_0 = constants['IV V']
K = constants['strike']
B = constants['barrier']
N = constants['no_samples']
tolerances = constants['tolerance range']
no_samples = N
weak_errors = []
ave_no_steps = []
no_refines = 3
two_to_refines = 2**no_refines

for c in tolerances:
    payoffs = []
    payoffs_fine = []
    no_steps = []
    for i in tqdm(range(no_samples)):
        y_output = [y_0]
        u_output = [u_0]
        dWv_crude_output = [0]
        dWu_crude_output = [0]
        h_crude = [0]
        y_fine_output = [y_0]
        u_fine_output = [u_0]
        dWv_fine_output = [0]
        dWu_fine_output = [0]
        step_counter = 0
        BMoutputs_v = BM(T, h_0)
        dWv_crude = BMoutputs_v['crude delta W']
        h = BMoutputs_v['step size']
        BMoutputs_u = BM(T, h_0)
        dWu_crude = BMoutputs_u['crude delta W']
        for i in range(len(dWv_crude)):
            y, u, dWv_crude_adapt, dWu_crude_adapt, h_new = milstein_adapt_iga(y_output[-1], u_output[-1], constants, c, h, dWv_crude[i], dWu_crude[i])
            y_output.extend(y)
            u_output.extend(u)
            dWv_crude_output.extend(dWv_crude_adapt)
            dWu_crude_output.extend(dWu_crude_adapt)
            h_crude.extend(h_new)
            step_counter += len(dWv_crude_adapt)
        times = [sum(h_crude[0:i+1]) for i in range(len(h_crude))]
        for i in range(1,len(dWv_crude_output)):
            dWv = dWv_crude_output[i]
            dWu = dWu_crude_output[i]
            h = h_crude[i]
            y_fine, u_fine, dWv_fine, dWu_fine = fine_step(y_fine_output[-1], u_fine_output[-1], constants, h, dWv, dWu, no_refines)
            y_fine_output.extend(y_fine)
            u_fine_output.extend(u_fine)
            dWv_fine_output.extend(dWv_fine)
            dWu_fine_output.extend(dWu_fine)
        h_fine = [0]*(two_to_refines*(len(h_crude)-1)+1)
        for i in range(len(h_crude)-1):
            h_fine[(two_to_refines*i)+1:two_to_refines*(i+1)+1] = [h_crude[i+1]/two_to_refines]*(two_to_refines)
        times_fine = [sum(h_fine[0:i+1]) for i in range(len(h_fine))]
        hit = 0
        for u in u_output:
            if np.exp(u) >= B:
                hit = 1
                break
        hit_fine = 0
        for u_fine in u_fine_output:
            if np.exp(u_fine) >= B:
                hit_fine = 1
                break
        if hit == 1:
            e_N = 0
        else:
            e_N = max(0, np.exp(u_output[-1])-K)

        if hit_fine == 1:
            e_N_fine = 0
        else:
            e_N_fine = max(0, np.exp(u_fine_output[-1])-K)
        payoffs.append(e_N)
        payoffs_fine.append(e_N_fine)
        no_steps.append(step_counter)
    mean_payoff = m.sqrt(stat.mean(payoffs))
    mean_payoff_fine = m.sqrt(stat.mean(payoffs_fine))
    weak_error = np.abs(mean_payoff - mean_payoff_fine)
    weak_errors.append(weak_error)
    ave_no_steps.append(stat.mean(no_steps))


## Define the function to create fine steps for Heun's

In [ ]:
def heun_fine_step(y, u, constants, h, dWv, dWu, no_refines = 3):
    sigma = constants['vol']
    mu = constants['drift']
    a_tilde = constants['a tilde']
    b_tilde = constants['b tilde']
    if no_refines == 0:
        y_tilde = y + a_tilde*(b_tilde - y)*h + sigma*y*dWv 
        y_new = y + (1/2)*h*(a_tilde*(b_tilde - y) + a_tilde*(b_tilde - y_tilde)) + (1/2)*(sigma*y + sigma*y_tilde)*dWv
        u_new = u + (mu - (1/4)*(y**2) - (1/4)*(y_tilde**2))*h + (1/2)*(y + y_tilde)*dWu
        return [y_new], [u_new], [dWv], [dWu]
    else:
        dWu_1, dWu_2, _ = refine(dWu, h)
        dWv_1, dWv_2, half_h_v = refine(dWv, h)
        y_new1, u_new1, dWv1 , dWu1= heun_fine_step(y, u, constants, half_h_v, dWv_1, dWu_1, no_refines - 1)
        y_new2, u_new2, dWv2, dWu2 = heun_fine_step(y_new1[-1], u_new1[-1], constants, half_h_v, dWv_2, dWu_2, no_refines - 1)

        return y_new1 + y_new2, u_new1 + u_new2, dWv1 + dWv2, dWu1 + dWu2

# Define the function for adaptive Heun's method

In [ ]:
def heun_adapt_iga(y, u, constants, tol, h, dWv, dWu,):
    sigma = constants['vol']
    mu = constants['drift']
    a_tilde = constants['a tilde']
    b_tilde = constants['b tilde']
    B = constants['barrier']
    c = tol

    if (B - np.exp(u) > c) or (h <= 1/512):
        y_tilde = y + a_tilde*(b_tilde - y)*h + sigma*y*dWv 
        y_new = y + (1/2)*h*(a_tilde*(b_tilde - y) + a_tilde*(b_tilde - y_tilde)) + (1/2)*(sigma*y + sigma*y_tilde)*dWv
        u_new = u + (mu - (1/4)*(y**2) - (1/4)*(y_tilde**2))*h + (1/2)*(y + y_tilde)*dWu
        return [y_new], [u_new], [dWv], [dWu], [h]
    else:
        dWu_1, dWu_2, _ = refine(dWu, h)
        dWv_1, dWv_2, half_h_v = refine(dWv, h)
        y_new1, u_new1, dWv1 , dWu1, h1 = heun_adapt_iga(y, u, constants, c, half_h_v, dWv_1, dWu_1)
        y_new2, u_new2, dWv2, dWu2, h2 = heun_adapt_iga(y_new1[-1], u_new1[-1], constants, c, half_h_v, dWv_2, dWu_2)
        return y_new1 + y_new2, u_new1 + u_new2, dWv1 + dWv2, dWu1 + dWu2, h1 + h2


## Run the Heun's method with adaptive step sizes for different tolerances 

In [ ]:
T = 1
h_0 = 1/2 * T
u_0 = constants['IV U']
y_0 = constants['IV V']
K = constants['strike']
B = constants['barrier']
N = constants['no_samples']
tolerances = constants['tolerance range']
no_samples = N
heun_weak_errors = []
heun_ave_no_steps = []
no_refines = 3
two_to_refines = 2**no_refines

for c in tolerances:
    heun_payoffs = []
    heun_payoffs_fine = []
    heun_no_steps = []
    for i in tqdm(range(no_samples)):
        y_output = [y_0]
        u_output = [u_0]
        dWv_crude_output = [0]
        dWu_crude_output = [0]
        h_crude = [0]
        y_fine_output = [y_0]
        u_fine_output = [u_0]
        dWv_fine_output = [0]
        dWu_fine_output = [0]
        step_counter = 0
        BMoutputs_v = BM(T, h_0)
        dWv_crude = BMoutputs_v['crude delta W']
        h = BMoutputs_v['step size']
        BMoutputs_u = BM(T, h_0)
        dWu_crude = BMoutputs_u['crude delta W']
        for i in range(len(dWv_crude)):
            y, u, dWv_crude_adapt, dWu_crude_adapt, h_new = heun_adapt_iga(y_output[-1], u_output[-1], constants, c, h, dWv_crude[i], dWu_crude[i])
            y_output.extend(y)
            u_output.extend(u)
            dWv_crude_output.extend(dWv_crude_adapt)
            dWu_crude_output.extend(dWu_crude_adapt)
            h_crude.extend(h_new)
            step_counter += len(dWv_crude_adapt)
        times = [sum(h_crude[0:i+1]) for i in range(len(h_crude))]
        for i in range(1,len(dWv_crude_output)):
            dWv = dWv_crude_output[i]
            dWu = dWu_crude_output[i]
            h = h_crude[i]
            y_fine, u_fine, dWv_fine, dWu_fine = heun_fine_step(y_fine_output[-1], u_fine_output[-1], constants, h, dWv, dWu, no_refines)
            y_fine_output.extend(y_fine)
            u_fine_output.extend(u_fine)
            dWv_fine_output.extend(dWv_fine)
            dWu_fine_output.extend(dWu_fine)
        h_fine = [0]*(two_to_refines*(len(h_crude)-1)+1)
        for i in range(len(h_crude)-1):
            h_fine[(two_to_refines*i)+1:two_to_refines*(i+1)+1] = [h_crude[i+1]/two_to_refines]*(two_to_refines)
        times_fine = [sum(h_fine[0:i+1]) for i in range(len(h_fine))]
        hit = 0
        for u in u_output:
            if np.exp(u) >= B:
                hit = 1
                break
        hit_fine = 0
        for u_fine in u_fine_output:
            if np.exp(u_fine) >= B:
                hit_fine = 1
                break
        if hit == 1:
            e_N = 0
        else:
            e_N = max(0, np.exp(u_output[-1])-K)

        if hit_fine == 1:
            e_N_fine = 0
        else:
            e_N_fine = max(0, np.exp(u_fine_output[-1])-K)
        heun_payoffs.append(e_N)
        heun_payoffs_fine.append(e_N_fine)
        heun_no_steps.append(step_counter)
    heun_mean_payoff = m.sqrt(stat.mean(heun_payoffs))
    heun_mean_payoff_fine = m.sqrt(stat.mean(heun_payoffs_fine))
    weak_error = np.abs(heun_mean_payoff - heun_mean_payoff_fine)
    heun_weak_errors.append(weak_error)
    heun_ave_no_steps.append(stat.mean(heun_no_steps))


## Plot error vs average number of steps for both methods

In [ ]:
from scipy.stats import linregress
mil_log_steps = [m.log10(x) for x in ave_no_steps]
mil_log_errors = [-m.log10(x) for x in weak_errors]
heun_log_steps = [m.log10(x) for x in heun_ave_no_steps]
heun_log_errors = [-m.log10(x) for x in heun_weak_errors]
plt.plot(mil_log_steps, mil_log_errors, label='Milstein',linestyle='-', marker="s", color='darkorange')
plt.plot(heun_log_steps, heun_log_errors, label='Heun', linestyle='dotted', marker="X", color='forestgreen')
plt.xlabel('log(ave no. steps)')
plt.ylabel('-log(E_N)')
plt.legend()
plt.grid(True)
slope_mil, intercept_mil, _, _, _ = linregress(mil_log_steps, mil_log_errors)
slope_heun, intercept_heun, _, _, _ = linregress(heun_log_steps, heun_log_errors)
print("Milstein:", slope_mil)
print("Heun:", slope_heun)